<a href="https://colab.research.google.com/github/notSURZO/LLM-Compression-Test/blob/main/Trying_Langchain_Vector_Stores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install langchain langchain-community chromadb sentence-transformers torch transformers accelerate tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00


In [8]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

texts = [
    "প্যারিসে আইফেল টাওয়ার অবস্থিত।",
    "চীনের মহাপ্রাচীর মহাকাশ থেকেও দৃশ্যমান।",
    "আমাজন রেইনফরেস্ট বিশ্বে বিশ শতাংশ অক্সিজেন সরবরাহ করে।"
]

embedding_model = HuggingFaceEmbeddings(
    model_name="l3cube-pune/bengali-sentence-similarity-sbert",
    model_kwargs={"device": "cpu"}  # or "cpu"
)

# Store texts with Chroma
db = Chroma.from_texts(texts=texts, embedding=embedding_model)

query = "আইফেল টাওয়ার কোথায়?"

# Get raw scores
raw_scores = db.similarity_search_with_score(query)

# Get relevance scores
relevance_scores = db.similarity_search_with_relevance_scores(query)

# Compare outputs
print("--- Raw Scores (Cosine Distance) ---")
for doc, score in raw_scores:
    print(f"Text: {doc.page_content}\nScore: {score:.4f}\n")

print("--- Relevance Scores (Normalized Cosine Similarity) ---")
for doc, score in relevance_scores:
    print(f"Text: {doc.page_content}\nRelevance Score: {score:.4f}\n")


--- Raw Scores (Cosine Distance) ---
Text: প্যারিসে আইফেল টাওয়ার অবস্থিত।
Score: 0.0754

Text: প্যারিসে আইফেল টাওয়ার অবস্থিত।
Score: 0.0754

Text: চীনের মহাপ্রাচীর মহাকাশ থেকেও দৃশ্যমান।
Score: 0.2804

Text: চীনের মহাপ্রাচীর মহাকাশ থেকেও দৃশ্যমান।
Score: 0.2804

--- Relevance Scores (Normalized Cosine Similarity) ---
Text: প্যারিসে আইফেল টাওয়ার অবস্থিত।
Relevance Score: 0.9467

Text: প্যারিসে আইফেল টাওয়ার অবস্থিত।
Relevance Score: 0.9467

Text: চীনের মহাপ্রাচীর মহাকাশ থেকেও দৃশ্যমান।
Relevance Score: 0.8017

Text: চীনের মহাপ্রাচীর মহাকাশ থেকেও দৃশ্যমান।
Relevance Score: 0.8017



In [9]:
pip install sentence-transformers torch numpy


In [13]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load model
model = SentenceTransformer("l3cube-pune/bengali-sentence-similarity-sbert", device="cpu")  # or "cpu"

# Define texts
texts = [
    "প্যারিসে আইফেল টাওয়ার অবস্থিত।",
    "চীনের মহাপ্রাচীর মহাকাশ থেকেও দৃশ্যমান।",
    "আমাজন রেইনফরেস্ট বিশ্বে বিশ শতাংশ অক্সিজেন সরবরাহ করে।"
]

# Define query
query = "আইফেল টাওয়ার কোথায়?"

# Encode without normalization to keep L2 distance meaningful
text_embeddings = model.encode(texts, convert_to_tensor=True, normalize_embeddings=False)
query_embedding = model.encode(query, convert_to_tensor=True, normalize_embeddings=False)

# Cosine similarity (optional: normalize before computing)
cos_sim = torch.nn.functional.cosine_similarity(
    torch.nn.functional.normalize(query_embedding, dim=0),
    torch.nn.functional.normalize(text_embeddings, dim=1)
)

# L2 distance
l2_distances = torch.norm(text_embeddings - query_embedding, dim=1)

# Display results
for i in range(len(texts)):
    print(f"Text: {texts[i]}")
    print(f"Cosine Similarity: {cos_sim[i].item():.4f}")
    print(f"L2 Distance: {l2_distances[i].item():.4f}\n")


Text: প্যারিসে আইফেল টাওয়ার অবস্থিত।
Cosine Similarity: 0.7873
L2 Distance: 0.2747

Text: চীনের মহাপ্রাচীর মহাকাশ থেকেও দৃশ্যমান।
Cosine Similarity: 0.1835
L2 Distance: 0.5296

Text: আমাজন রেইনফরেস্ট বিশ্বে বিশ শতাংশ অক্সিজেন সরবরাহ করে।
Cosine Similarity: 0.0256
L2 Distance: 0.6037



In [17]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import torch

# Initialize your HuggingFace embedding model
model = SentenceTransformer("l3cube-pune/bengali-sentence-similarity-sbert", device="cpu")  # or "cpu"

# Setup Chroma in-memory DB
chroma_client = chromadb.Client()

# Create a collection
collection = chroma_client.create_collection(name="my_docs")

# Sample texts to store
texts = [
    "প্যারিসে আইফেল টাওয়ার অবস্থিত।",
    "চীনের মহাপ্রাচীর মহাকাশ থেকেও দৃশ্যমান।",
    "আমাজন রেইনফরেস্ট বিশ্বে বিশ শতাংশ অক্সিজেন সরবরাহ করে।"
]

# Create embeddings for texts
embeddings = model.encode(texts, normalize_embeddings=False).tolist()  # Do NOT normalize for L2

# Add to Chroma
collection.add(
    documents=texts,
    embeddings=embeddings,
    ids=[f"doc{i}" for i in range(len(texts))]
)

# Now do a query
query = "আইফেল টাওয়ার কোথায়?"
query_embedding = model.encode([query], normalize_embeddings=False).tolist()[0]

# Perform similarity search
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3,
    include=["distances", "embeddings", "documents", "metadatas"]
)

# Print results
for doc, distance in zip(results["documents"][0], results["distances"][0]):
    print(f"Doc: {doc}")
    print(f"Distance: {distance:.4f}\n")


Doc: প্যারিসে আইফেল টাওয়ার অবস্থিত।
Distance: 0.0754

Doc: চীনের মহাপ্রাচীর মহাকাশ থেকেও দৃশ্যমান।
Distance: 0.2804

Doc: আমাজন রেইনফরেস্ট বিশ্বে বিশ শতাংশ অক্সিজেন সরবরাহ করে।
Distance: 0.3644

